# Getting seasonal forecast data

In [1]:
options(java.parameters = "-Xmx100000m")
library(transformeR)
library(loadeR.ECOMS)
library(loadeR)
library(visualizeR)
library(convertR)
library(drought4R)
library(downscaleR)

install.packages("rlist")
library("rlist")

transformeR version 1.5.1 (2019-07-13) is loaded
Get the latest stable version (1.6.1) using <devtools::install_github('SantanderMetGroup/transformeR')>
Please see 'citation("transformeR")' to cite this package.
Loading required package: loadeR
Loading required package: rJava
Loading required package: loadeR.java
Java version 1.8x amd64 by Azul Systems, Inc. detected
NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready
loadeR version 1.4.15 (2019-07-30) is loaded
Please use 'citation("loadeR")' to cite this package.
loadeR.ECOMS version 1.4.6 (2018-10-20) is loaded
Please use 'citation("loadeR.ECOMS")' to cite this package.
Warning message:
“no DISPLAY variable so Tk is not available”visualizeR version 1.4.0 (2019-07-09) is loaded
Get the latest stable version (1.5.0) using <devtools::install_github('SantanderMetGroup/visualizeR')>
Please see 'citation("visualizeR")' to cite this package.
Loading required package: udunits2
udunits system database read from /opt/conda/lib/

In [2]:
# Output path where the data will be saved (change to your local path).
dir.data <- './data/' 
dir.Rdata <- './Rdata/'


#Defining the geographical domain
lonLim <- c(8, 13) 
latLim <- c(57, 62)

# Define the period and the season
years <- 1979:2019
season <- 1:12 #Full year

# Location of the lake
lake <- list(x= 10.7438771059,  y= 59.4392814052 )
lakename <- 'Vansjo'
    

# Define the dataset 
dataset <- "System4_seasonal_15"

loadeR::loginUDG(username='WATExR',password="1234567890")

# Check available variables in the dataset 
di <- dataInventory("http://www.meteo.unican.es/tds5/dodsC/system4/System4_Seasonal_15Members.ncml")
names(di)

[2019-11-13 11:05:40] Setting credentials...
[2019-11-13 11:05:41] Success!
Go to <http://www.meteo.unican.es/udg-tap/home> for details on your authorized groups and datasets
[2019-11-13 11:05:41] Doing inventory ...
[2019-11-13 11:05:48] Retrieving info for 'z1000mb' (45 vars remaining)
[2019-11-13 11:05:50] Retrieving info for 'z700mb' (44 vars remaining)
[2019-11-13 11:05:50] Retrieving info for 'z850mb' (43 vars remaining)
[2019-11-13 11:05:51] Retrieving info for 'z500mb' (42 vars remaining)
[2019-11-13 11:05:51] Retrieving info for 'z300mb' (41 vars remaining)
[2019-11-13 11:05:51] Retrieving info for 'z200mb' (40 vars remaining)
[2019-11-13 11:05:52] Retrieving info for 't1000mb' (39 vars remaining)
[2019-11-13 11:05:52] Retrieving info for 't700mb' (38 vars remaining)
[2019-11-13 11:05:53] Retrieving info for 't850mb' (37 vars remaining)
[2019-11-13 11:05:53] Retrieving info for 't500mb' (36 vars remaining)
[2019-11-13 11:05:53] Retrieving info for 't300mb' (35 vars remaining)


[1] "z1000mb"  "z700mb"   "z850mb"   "z500mb"   "z300mb"   "z200mb"  
 [7] "t1000mb"  "t700mb"   "t850mb"   "t500mb"   "t300mb"   "t200mb"  
[13] "u1000mb"  "u700mb"   "u850mb"   "u500mb"   "u300mb"   "u200mb"  
[19] "v1000mb"  "v700mb"   "v850mb"   "v500mb"   "v300mb"   "v200mb"  
[25] "q1000mb"  "q700mb"   "q850mb"   "q500mb"   "q300mb"   "q200mb"  
[31] "zsfc"     "ssrd"     "strd"     "tcc"      "u10m"     "v10m"    
[37] "dpt2m"    "t2m"      "mean2t24" "sst"      "mx2t24"   "mn2t24"  
[43] "sd"       "sf"       "tp"       "mslp"

In [3]:
# Path to the observational data (change to your local path).
dir.Rdata.obs <- "./Rdata/PIK_Obs-EWEMBI_1_2_3_4_5_6_7_8_9_10_11_12_uas_vas_ps_tas_pr_rsds_rlds_hurs_cc.rda"
obs.data <- get(load(dir.Rdata.obs))

In [4]:
# Define the variables to be loaded (the same as in the observational data, 
# except clould cover (cc) and evapotranspiration (petH))
sapply(obs.data, function(x) getVarNames(x)) # to check the variables in the observational data.
variables <- c("uas", "vas", "ps", "tas", "pr", "rsds", "rlds", "hurs")

# Define daily aggregation function for each variable selected
aggr.fun <- c("mean", "mean", "mean", "mean", "sum", "mean", "mean", "mean")

# Define the members
mem <- 1:15
# Define the lead month
lead.month <- 0
# Define period and season
years <- 1981:2010

uas    vas     ps    tas     pr   rsds   rlds   hurs     cc 
 "uas"  "vas"   "ps"  "tas"   "pr" "rsds" "rlds" "hurs"   "cc"

In [5]:
########## DATA LOADING AND TRANSFORMATION ----------------------------------------------------------

# Load seasonal forecast data (System4 or CFS) with function loadECOMS
# Data is loaded in a loop (función lapply) to load all variables in a single code line.
# A list of grids is obtained, each slot in the list corresponds to a variable


In [6]:
# season <- c(2,3,4,5) # Spring
# rm(data)
# # Load seasonal forecast data (System4 or CFS) with function loadECOMS
# # Data is loaded in a loop (función lapply) to load all variables in a single code line.
# # A list of grids is obtained, each slot in the list corresponds to a variable
# data.prelim <- lapply(1:length(variables), function(x) loadECOMS(dataset, var = variables[x], years = years, 
#                                                           members = mem, leadMonth = lead.month,
#                                                           lonLim = lonLim, latLim = latLim, season = season, 
#                                                           time = "DD", aggr.d = aggr.fun[x]))
# names(data.prelim) <- variables

# # Bilinear interpolation of the data to the location of the lake
# data.interp <- lapply(data.prelim, function(x) interpGrid(x, new.coordinates = lake, 
#                                                           method = "bilinear", 
#                                                           bilin.method = "akima"))

# # Convert pressure units to millibars with function udConvertGrid from package convertR.
# data.interp$ps <- udConvertGrid(data.interp$ps, new.units = "millibars")

# # Compute cloud cover with function rad2cc
# clt <- rad2cc(rsds = data.interp$rsds, rlds = data.interp$rlds)
# clt$Variable$varName <- "cc"

# # Put all variables together
# data <- c(data.interp, "cc" = list(clt))
# list.save(data, "era_system4_spring.rds")

In [7]:
# season <- c(5,6,7,8) # Summer
# rm(data)
# # Load seasonal forecast data (System4 or CFS) with function loadECOMS
# # Data is loaded in a loop (función lapply) to load all variables in a single code line.
# # A list of grids is obtained, each slot in the list corresponds to a variable
# data.prelim <- lapply(1:length(variables), function(x) loadECOMS(dataset, var = variables[x], years = years, 
#                                                           members = mem, leadMonth = lead.month,
#                                                           lonLim = lonLim, latLim = latLim, season = season, 
#                                                           time = "DD", aggr.d = aggr.fun[x]))
# names(data.prelim) <- variables

# # Bilinear interpolation of the data to the location of the lake
# data.interp <- lapply(data.prelim, function(x) interpGrid(x, new.coordinates = lake, 
#                                                           method = "bilinear", 
#                                                           bilin.method = "akima"))

# # Convert pressure units to millibars with function udConvertGrid from package convertR.
# data.interp$ps <- udConvertGrid(data.interp$ps, new.units = "millibars")

# # Compute cloud cover with function rad2cc
# clt <- rad2cc(rsds = data.interp$rsds, rlds = data.interp$rlds)
# clt$Variable$varName <- "cc"

# # Put all variables together
# data <- c(data.interp, "cc" = list(clt))
# list.save(data, "era_system4_summer.rds")                      

In [8]:
# season <- c(8,9,10,11) # Fall
# rm(data)
# # Load seasonal forecast data (System4 or CFS) with function loadECOMS
# # Data is loaded in a loop (función lapply) to load all variables in a single code line.
# # A list of grids is obtained, each slot in the list corresponds to a variable
# data.prelim <- lapply(1:length(variables), function(x) loadECOMS(dataset, var = variables[x], years = years, 
#                                                           members = mem, leadMonth = lead.month,
#                                                           lonLim = lonLim, latLim = latLim, season = season, 
#                                                           time = "DD", aggr.d = aggr.fun[x]))
# names(data.prelim) <- variables

# # Bilinear interpolation of the data to the location of the lake
# data.interp <- lapply(data.prelim, function(x) interpGrid(x, new.coordinates = lake, 
#                                                           method = "bilinear", 
#                                                           bilin.method = "akima"))

# # Convert pressure units to millibars with function udConvertGrid from package convertR.
# data.interp$ps <- udConvertGrid(data.interp$ps, new.units = "millibars")

# # Compute cloud cover with function rad2cc
# clt <- rad2cc(rsds = data.interp$rsds, rlds = data.interp$rlds)
# clt$Variable$varName <- "cc"

# # Put all variables together
# data <- c(data.interp, "cc" = list(clt))
# list.save(data, "era_system4_fall.rds")

In [9]:
# season <- c(11,12,1,2) # Winter
# rm(data)

# data.prelim <- lapply(1:length(variables), function(x) loadECOMS(dataset, var = variables[x], years = years, 
#                                                           members = mem, leadMonth = lead.month,
#                                                           lonLim = lonLim, latLim = latLim, season = season, 
#                                                           time = "DD", aggr.d = aggr.fun[x]))
# names(data.prelim) <- variables

# # Bilinear interpolation of the data to the location of the lake
# data.interp <- lapply(data.prelim, function(x) interpGrid(x, new.coordinates = lake, 
#                                                           method = "bilinear", 
#                                                           bilin.method = "akima"))

# # Convert pressure units to millibars with function udConvertGrid from package convertR.
# data.interp$ps <- udConvertGrid(data.interp$ps, new.units = "millibars")

# # Compute cloud cover with function rad2cc
# clt <- rad2cc(rsds = data.interp$rsds, rlds = data.interp$rlds)
# clt$Variable$varName <- "cc"

# # Put all variables together
# data <- c(data.interp, "cc" = list(clt))
# list.save(data, "era_system4_winter.rds")

## Bias correction

In [10]:
##### BIAS CORRECTION -----------------------------------------------------------------------
# Subset all datasets to the same Dates as the hindcast precipitation. Note that we compute daily accumulated 
# precipitation, for this reason this variable has no value for the first day of every season.  
data = list.load ("era_system4_winter.rds")
if (sum(names(data)=="pr")>0){
  data <- lapply(1:length(data), function(x)  {intersectGrid(data[[x]], data[[which(names(data)=="pr")]], type = "temporal", which.return = 1)}) 
  names(data) <- sapply(data, function(x) getVarNames(x))
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))
} else{
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))  
}

# Check variable consistency
if (!identical(names(obs.data), names(data))) stop("variables in obs.data and data (seasonal forecast) do not match.")

# Bias correction with leave-one-year-out ("loo") cross-validation
# type ?biasCorrection in R for more info about the parameter settings for bias correction.
data.bc.cross <- lapply(1:length(data), function(x)  {
    precip <- FALSE
    #if (names(data)[x] == "pr") precip <- TRUE
    biasCorrection(y = obs.data[[x]], x = data[[x]], 
                method = "eqm", cross.val = "loo",
                precipitation = precip,
                wet.threshold = 1,
                join.members = TRUE)
  }) 
names(data.bc.cross) <- names(data)
# Bias correction without cross-validation
data.bc <- lapply(1:length(data), function(v)  {
    pre <- FALSE
    #if (names(data)[v] == "pr") pre <- TRUE
    biasCorrection(y = obs.data[[v]], x = data[[v]], 
                 method = "eqm",
                 precipitation = pre,
                 wet.threshold = 1,
                 join.members = TRUE)
}) 
names(data.bc) <- names(data)  
                            
list.save(data.bc.cross, "era_system4_winter_bias_corrected_cross_no_precip.rds")
list.save(data.bc, "era_system4_winter_bias_corrected_no_precip.rds")
# save Rdata (*.rda file)
#save(data, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_raw.rda"))
#save(data.bc.cross, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_BCcross.rda"))
#save(data.bc, file = paste0(dir.Rdata, dataset, 

Validation 1, 28 remaining
[2019-11-13 10:23:15] Argument precipitation is set as FALSE, please ensure that this matches your data.
[2019-11-13 10:23:18] Number of windows considered: 1...
[2019-11-13 10:23:18] Bias-correcting 15 members considering their joint distribution...
[2019-11-13 10:23:18] Done.
Validation 2, 27 remaining
[2019-11-13 10:23:22] Argument precipitation is set as FALSE, please ensure that this matches your data.
[2019-11-13 10:23:25] Number of windows considered: 1...
[2019-11-13 10:23:25] Bias-correcting 15 members considering their joint distribution...
[2019-11-13 10:23:26] Done.
Validation 3, 26 remaining
[2019-11-13 10:23:29] Argument precipitation is set as FALSE, please ensure that this matches your data.
[2019-11-13 10:23:32] Number of windows considered: 1...
[2019-11-13 10:23:32] Bias-correcting 15 members considering their joint distribution...
[2019-11-13 10:23:32] Done.
Validation 4, 25 remaining
[2019-11-13 10:23:35] Argument precipitation is set as 

In [5]:
##### BIAS CORRECTION -----------------------------------------------------------------------
# Subset all datasets to the same Dates as the hindcast precipitation. Note that we compute daily accumulated 
# precipitation, for this reason this variable has no value for the first day of every season.  
data = list.load ("era_system4_fall.rds")
if (sum(names(data)=="pr")>0){
  data <- lapply(1:length(data), function(x)  {intersectGrid(data[[x]], data[[which(names(data)=="pr")]], type = "temporal", which.return = 1)}) 
  names(data) <- sapply(data, function(x) getVarNames(x))
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))
} else{
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))  
}

# Check variable consistency
if (!identical(names(obs.data), names(data))) stop("variables in obs.data and data (seasonal forecast) do not match.")

# Bias correction with leave-one-year-out ("loo") cross-validation
# type ?biasCorrection in R for more info about the parameter settings for bias correction.
data.bc.cross <- lapply(1:length(data), function(x)  {
    precip <- FALSE
    #if (names(data)[x] == "pr") precip <- TRUE
    biasCorrection(y = obs.data[[x]], x = data[[x]], 
                method = "eqm", cross.val = "loo",
                precipitation = precip,
                wet.threshold = 1,
                join.members = TRUE)
  }) 
names(data.bc.cross) <- names(data)
# Bias correction without cross-validation
data.bc <- lapply(1:length(data), function(v)  {
    pre <- FALSE
    #if (names(data)[v] == "pr") pre <- TRUE
    biasCorrection(y = obs.data[[v]], x = data[[v]], 
                 method = "eqm",
                 precipitation = pre,
                 wet.threshold = 1,
                 join.members = TRUE)
}) 
names(data.bc) <- names(data)  
                            
list.save(data.bc.cross, "era_system4_fall_bias_corrected_cross_no_precip.rds")
list.save(data.bc, "era_system4_fall_bias_corrected_no_precip.rds")

# save Rdata (*.rda file)
#save(data, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_raw.rda"))
#save(data.bc.cross, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_BCcross.rda"))
#save(data.bc, file = paste0(dir.Rdata, dataset, 

Validation 1, 29 remaining
[2019-11-13 11:06:09] Argument precipitation is set as FALSE, please ensure that this matches your data.
[2019-11-13 11:06:12] Number of windows considered: 1...
[2019-11-13 11:06:12] Bias-correcting 15 members considering their joint distribution...
[2019-11-13 11:06:13] Done.
Validation 2, 28 remaining
[2019-11-13 11:06:17] Argument precipitation is set as FALSE, please ensure that this matches your data.
[2019-11-13 11:06:21] Number of windows considered: 1...
[2019-11-13 11:06:21] Bias-correcting 15 members considering their joint distribution...
[2019-11-13 11:06:21] Done.
Validation 3, 27 remaining
[2019-11-13 11:06:24] Argument precipitation is set as FALSE, please ensure that this matches your data.
[2019-11-13 11:06:27] Number of windows considered: 1...
[2019-11-13 11:06:27] Bias-correcting 15 members considering their joint distribution...
[2019-11-13 11:06:28] Done.
Validation 4, 26 remaining
[2019-11-13 11:06:31] Argument precipitation is set as 

In [ ]:
##### BIAS CORRECTION -----------------------------------------------------------------------
# Subset all datasets to the same Dates as the hindcast precipitation. Note that we compute daily accumulated 
# precipitation, for this reason this variable has no value for the first day of every season.  
data = list.load ("era_system4_spring.rds")
if (sum(names(data)=="pr")>0){
  data <- lapply(1:length(data), function(x)  {intersectGrid(data[[x]], data[[which(names(data)=="pr")]], type = "temporal", which.return = 1)}) 
  names(data) <- sapply(data, function(x) getVarNames(x))
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))
} else{
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))  
}

# Check variable consistency
if (!identical(names(obs.data), names(data))) stop("variables in obs.data and data (seasonal forecast) do not match.")

# Bias correction with leave-one-year-out ("loo") cross-validation
# type ?biasCorrection in R for more info about the parameter settings for bias correction.
data.bc.cross <- lapply(1:length(data), function(x)  {
    precip <- FALSE
    #if (names(data)[x] == "pr") precip <- TRUE
    biasCorrection(y = obs.data[[x]], x = data[[x]], 
                method = "eqm", cross.val = "loo",
                precipitation = precip,
                wet.threshold = 1,
                join.members = TRUE)
  }) 
names(data.bc.cross) <- names(data)
# Bias correction without cross-validation
data.bc <- lapply(1:length(data), function(v)  {
    pre <- FALSE
    #if (names(data)[v] == "pr") pre <- TRUE
    biasCorrection(y = obs.data[[v]], x = data[[v]], 
                 method = "eqm",
                 precipitation = pre,
                 wet.threshold = 1,
                 join.members = TRUE)
}) 
names(data.bc) <- names(data)  
                            
list.save(data.bc.cross, "era_system4_spring_bias_corrected_cross_no_precip.rds")
list.save(data.bc, "era_system4_spring_bias_corrected_no_precip.rds")
# save Rdata (*.rda file)
#save(data, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_raw.rda"))
#save(data.bc.cross, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_BCcross.rda"))
#save(data.bc, file = paste0(dir.Rdata, dataset, 

In [ ]:
##### BIAS CORRECTION -----------------------------------------------------------------------
# Subset all datasets to the same Dates as the hindcast precipitation. Note that we compute daily accumulated 
# precipitation, for this reason this variable has no value for the first day of every season.  
data = list.load ("era_system4_summer.rds")
if (sum(names(data)=="pr")>0){
  data <- lapply(1:length(data), function(x)  {intersectGrid(data[[x]], data[[which(names(data)=="pr")]], type = "temporal", which.return = 1)}) 
  names(data) <- sapply(data, function(x) getVarNames(x))
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))
} else{
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))  
}

# Check variable consistency
if (!identical(names(obs.data), names(data))) stop("variables in obs.data and data (seasonal forecast) do not match.")

# Bias correction with leave-one-year-out ("loo") cross-validation
# type ?biasCorrection in R for more info about the parameter settings for bias correction.
data.bc.cross <- lapply(1:length(data), function(x)  {
    precip <- FALSE
    #if (names(data)[x] == "pr") precip <- TRUE
    biasCorrection(y = obs.data[[x]], x = data[[x]], 
                method = "eqm", cross.val = "loo",
                precipitation = precip,
                wet.threshold = 1,
                join.members = TRUE)
  }) 
names(data.bc.cross) <- names(data)
# Bias correction without cross-validation
data.bc <- lapply(1:length(data), function(v)  {
    pre <- FALSE
    #if (names(data)[v] == "pr") pre <- TRUE
    biasCorrection(y = obs.data[[v]], x = data[[v]], 
                 method = "eqm",
                 precipitation = pre,
                 wet.threshold = 1,
                 join.members = TRUE)
}) 
names(data.bc) <- names(data)  
                            
list.save(data.bc.cross, "era_system4_summer_bias_corrected_cross_no_precip.rds")
list.save(data.bc, "era_system4_summer_bias_corrected_no_precip.rds")

# save Rdata (*.rda file)
#save(data, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_raw.rda"))
#save(data.bc.cross, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_BCcross.rda"))
#save(data.bc, file = paste0(dir.Rdata, dataset, 